仕様：
- デフォルトの行列は(3^8, 3^8)型
- dimという文字列は3のこと。つまりdim\*\*8という文字列がよく登場することになる。
- global変数はOmega_global,B_global,m_global  

(3,...)と(3^8,3^8)の変換ルール：
- 行列$A$の$\ket{j_1}\bra{j_2}$成分がndarrayでのA[j1, j2]に対応(ndarrayを行列と見た時の普通の解釈と同じ)
- ndarrayのテンソル積は$A\otimes B$の$\ket{j_1}\bra{j_2}\otimes \ket{j_3}\bra{j_4}$成分がndarrayで対応するAとBをテンソル積して得たndarray C (shape: (dim, dim, dim, dim))の[j1, j2, j3, j4]成分
- shape:(dim, )\*(2\*N)とshape:(dim^N, dim^N)のndarrayの対応は、行列においてのテンソル積$A\otimes B$の解釈と合うようにとった。具体的には、shape: (dim, dim, dim, dim)のCとshape: (dim\*\*2, dim\*\*2)のC'のndarrayの対応が、C[j1, j2, j3, j4]=C'[ dim\*j1+j3, dim\*j2+j4]となるようにした。

設計図:  
- Omega_globalの行列生成関数(ランダムユニタリを5\*10^5回samplingして足す, 最後に(3,...)から(3^8, 3^8)に変換)  
- Q_combの関数構築(部分traceからのtensorをかけるみたいな。入力(3^8,3^8)→(3,...)→出力(3^8,3^8))  
- Tの関数構築 → Q_primeの関数構築  
- Omega_primeに比例する成分を取り出す関数(Omega_prime_proj)構築(global Omega_globalを使う)  
- Bの行列構築関数 (結構大変、H_1, H_2をrandomに選ぶ → B_pre作成 → Omega_prime_projを用いてB_global作成)  
- B_primeに比例する成分を取り出す関数(B_prime_proj)構築(global B_globalを使う)
- 関数Qを構築(入力(3^8,3^8)→出力(3^8,3^8))  
- check3を微妙に変えて(def Q変更, normはそのまま使えそう、vec2mat等のdim\*\*2をdim\*\*8に変更、またlower_eig1等のdimを引数に取る関数の中身も変えた方が良い)、Omega_globalとB_globalを生成してからcheck3の計算実行


In [ ]:
import numpy as np
import time as time
from scipy import optimize

def HS_ip(A,B): #shape=(D,D)を想定
    return (np.multiply(A.conj(), B)).sum()

def norm(A):
    return np.sqrt(HS_ip(A,A))

def normalize(A):
    return A/norm(A)

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x)>= 0)

In [ ]:
arr = np.random.uniform(-1, 1, (3,3)).astype(complex) + 1.0j * np.random.uniform(-1, 1, (3,3)).astype(complex)
brr = np.random.uniform(-1, 1, (3,3)).astype(complex) + 1.0j * np.random.uniform(-1, 1, (3,3)).astype(complex)
print("HS_ip working?:{}".format(HS_ip(arr, brr) - np.trace((arr.conj().T)@brr)<=1e-10))
print("norm working?:{}".format(norm(arr)-np.trace(arr.conj().T@arr)<=1e-10))
print("normalize working?:{}".format(np.trace(normalize(arr).conj().T@normalize(arr))-1<=1e-10))

In [ ]:
dim = 3

def tens_prod(*args):
    if len(args)==2:
        a=args[0]
        b=args[1]
        assert (type(a) == np.ndarray) and (type(b) == np.ndarray), "input has to be ndarray"
        return np.tensordot(a, b, axes=0)
    a=args[:-1]
    b=args[-1]
    return tens_prod(tens_prod(*a), b)

def partial_trace(mat, dim, N_tens, syss):
    assert mat.shape == (dim,) * (2 * N_tens), "matrix shape mismatch"
    if not hasattr(syss, "__len__"):
        syss = [syss]
    mout = mat
    syss.sort()
    for i in range(len(syss)):
        mout = np.trace(mout, axis1=(syss[i]-i)*2, axis2=(syss[i]-i)*2+1)
    return mout

def partial_red(mat, dim, N_tens, syss):
    assert mat.shape == (dim,) * (2 * N_tens), "matrix shape mismatch"
    if not hasattr(syss, "__len__"):
        syss = [syss]
    mout = mat
    syss.sort()
    for i in range(len(syss)):
        mout = np.trace(mout, axis1=(syss[i])*2, axis2=(syss[i])*2+1)
        mout = tens_prod(mout, np.eye(dim)/dim)
        l_pre = list(range(syss[i]*2))
        l_mid = [N_tens*2-2, N_tens*2-1]
        l_post = list(range(syss[i]*2, N_tens*2-2))
        try:
            mout = np.transpose(mout, l_pre+l_mid+l_post)
        except:
            print("list:{}, shape:{}".format(l_pre+l_mid+l_post, mout.shape))
            raise Exception
    return mout

arr0 = np.arange(dim**2).reshape(dim,dim)
arr1 = np.arange(dim**2, 2*dim**2).reshape(dim,dim)
arr2 = np.arange(2*dim**2, 3*dim**2).reshape(dim,dim)
arr3 = np.arange(3*dim**2, 4*dim**2).reshape(dim,dim)

l = [1,2,0,2,1,0,1,2]
print("A[{},{}]:{}, B[{},{}]:{}, C[{},{}]:{}, D[{},{}]:{}".format(\
    l[0],l[1],arr0[l[0],l[1]],l[2],l[3],arr1[l[2],l[3]],l[4],l[5],arr2[l[4],l[5]],l[6],l[7],arr3[l[6],l[7]]))
t0123 = tens_prod(arr0, arr1, arr2, arr3)
print("t[{},{},{},{},{},{},{},{}]:{}".format(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],t0123[l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7]]))
print("A[{0},{1}]*B[{2},{3}]*C[{4},{5}]*D[{6},{7}]=t0123[{0},{1},{2},{3},{4},{5},{6},{7}]:{8}".format(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],arr0[l[0],l[1]]*arr1[l[2],l[3]]*arr2[l[4],l[5]]*arr3[l[6],l[7]]==t0123[l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7]]))

t13 = tens_prod(arr1, arr3)
t13_test = partial_trace(t0123, dim, 4, [2,0]) / (np.trace(arr0)*np.trace(arr2))
print("partial trace working?:",np.all(t13 == t13_test))

red02t = partial_red(t0123, 3, 4, [2,0])
red02t_actual = tens_prod(np.eye(dim)/dim, arr1, np.eye(dim)/dim, arr3)* \
                np.trace(arr0) * np.trace(arr2)
print("partial reduction working?",np.allclose(red02t, red02t_actual, 0, 1e-8))

def manysys2default(mat, dim, N_tens):
    assert mat.shape == (dim,) * (2 * N_tens), "matrix shape mismatch"
    tup = tuple(range(0,2*N_tens,2)) + tuple(range(1,2*N_tens+1,2))
    return mat.transpose(tup).reshape((dim**N_tens, dim**N_tens))

def default2manysys(mat, dim, N_tens):
    assert mat.shape == (dim**N_tens, dim**N_tens), "matrix shape mismatch"
    tup = tuple([(k//2)+(k%2)*N_tens for k in range(2*N_tens)])
    return mat.reshape((dim,)*(2*N_tens)).transpose(tup)

N_tens = 4
t0123_def = manysys2default(t0123,dim,N_tens)
t0123_test = default2manysys(t0123_def, dim, N_tens)
tdef_ = np.random.uniform(-1,1,(dim**N_tens, dim**N_tens))
tdef_ma = default2manysys(tdef_, dim, N_tens)
tdef_test = manysys2default(tdef_ma,dim,N_tens)

print("manysys vs default transformation working?",np.all(t0123==t0123_test) and np.all(tdef_==tdef_test))

In [ ]:
def uni_Choi(unitary, dim):
    assert unitary.shape==(dim, dim), "dimension mismatch"
    arr = np.zeros((dim, dim, dim, dim)).astype(complex)
    for a in np.ndindex((dim, dim)):
        brr = np.zeros((dim, dim))
        brr[a] = 1
        brr = unitary @ brr @ unitary.conj().T
        arr[a] = brr
    return arr

In [ ]:
dim = 3
a = (np.random.uniform(-1,1,(dim, dim)).astype(complex) + 1.0j*np.random.uniform(-1,1,(dim, dim)).astype(complex))
b = (np.random.uniform(-1,1,(dim, dim)).astype(complex) + 1.0j*np.random.uniform(-1,1,(dim, dim)).astype(complex))
c = np.trace(np.tensordot(b.transpose(), uni_Choi(a,dim), axes=[1, 0]), axis1=0, axis2=1)
d = a@b@a.conj().T

print("uni_Choi working?:{}".format(np.allclose(c, d)))

In [ ]:
from scipy.stats import unitary_group, ortho_group

def gen_Omega2(dim, N_uni, in_style, out_style, N_sample):
    assert len(in_style) == N_uni-1, "N_uni must be total of numbers of input unitaries and the number of output unitary"
    # arr = np.zeros((dim,)*(4*N_uni)).astype(complex)
    N_rep = int(np.ceil(np.log2(N_sample)))
    print("N_rep:",N_rep)
    max_ent = uni_Choi(np.eye(dim),dim)
    mat = manysys2default(tens_prod(*((max_ent,)*N_uni)), dim, 2*N_uni) / (dim**2)
    for n in range(N_rep):
        print("iteration:", n+1)
        # uni1 = unitary_group.rvs(dim)
        # uni2 = unitary_group.rvs(dim)
        uni1 = ortho_group.rvs(dim)
        uni2 = ortho_group.rvs(dim)
        mat1 = manysys2default(tens_prod(*((np.eye(dim), uni1)*N_uni)),dim,2*N_uni)@mat@manysys2default(tens_prod(*((np.eye(dim), uni1.conj().T)*N_uni)),dim,2*N_uni)
        mat2 = manysys2default(tens_prod(*((np.eye(dim), uni2)*N_uni)),dim,2*N_uni)@mat@manysys2default(tens_prod(*((np.eye(dim), uni2.conj().T)*N_uni)),dim,2*N_uni)
        mat = (mat1 + mat2) / 2

    mat = default2manysys(mat, dim, 2*N_uni)
    for j in range(N_uni):
        if j <= N_uni-2:
            if in_style[j] == "identity": #cc
                tup = tuple([k if (k not in range(4*j, 4*j+4)) else (k//2)*2+(k+1)%2 for k in range(4*N_uni)])
                mat = mat.transpose(tup)
            elif in_style[j] == "cc": #I
                pass
            elif in_style[j] == "transpose": # dagger
                tup = tuple([k if (k not in range(4*j, 4*j+4)) else 8*j+3-k for k in range(4*N_uni)])
                mat = mat.transpose(tup)
            elif in_style[j] == "dagger": #tra
                tup =tuple([k if (k not in range(4*j, 4*j+4)) else (k//4)*4+(k+2)%4 for k in range(4*N_uni)])
                mat = mat.transpose(tup)
            else:
                print(in_style, in_style[j], j)
                raise Exception("in_style string not recognized")
        if j == N_uni-1:
            if out_style == "identity":
                pass
            elif out_style == "cc":
                tup = tuple([k if (k not in range(4*j, 4*j+4)) else (k//2)*2+(k+1)%2 for k in range(4*N_uni)])
                mat = mat.transpose(tup)
            elif out_style == "transpose":
                tup =tuple([k if (k not in range(4*j, 4*j+4)) else (k//4)*4+(k+2)%4 for k in range(4*N_uni)])
                mat = mat.transpose(tup)
            elif out_style == "dagger":
                tup = tuple([k if (k not in range(4*j, 4*j+4)) else 8*j+3-k for k in range(4*N_uni)])
                mat = mat.transpose(tup)
            else:
                raise Exception("out_style string not recognized")
    mat = mat.transpose((4*N_uni-4, 4*N_uni-3)+tuple(range(4*N_uni-4))+(4*N_uni-2, 4*N_uni-1))
    mat = manysys2default(mat, dim, 2*N_uni)
    return mat

# dim = 2
# N_uni = 5
# in_style = ["identity","identity","identity","identity"]
# out_style = "dagger"
# N_sample = 5*(10**5)

# dim = 2
# N_uni = 3
# in_style = ["identity","identity"]
# out_style = "dagger"
# N_sample = 5*(10**5)

# dim = 3
# N_uni = 4
# in_style = ["cc", "identity", "cc"]
# out_style = "transpose"
# N_sample = 5*(10**5)

dim = 3
N_uni = 3
in_style = ["identity", "identity"]
out_style = "dagger"
N_sample = 5*(10**5)

Omega_global = gen_Omega2(dim, N_uni, in_style, out_style, N_sample)
Omega_global.shape 

In [ ]:
print(np.trace(Omega_global)) 
print(Omega_global.shape)

In [ ]:
def Q_comb(M, dim, N_uni):
    MM = default2manysys(M, dim, 2*N_uni)
    pm = partial_red(MM, dim, 2*N_uni, [2*N_uni-1])-partial_red(MM, dim, 2*N_uni, [2*N_uni-1, 2*N_uni-2])
    for i in range(N_uni-1):
        pm += partial_red(MM, dim, 2*N_uni, list(range(2*N_uni-3-2*i, 2*N_uni))) \
        - partial_red(MM, dim, 2*N_uni, list(range(2*N_uni-4-2*i, 2*N_uni)))
    return manysys2default(pm, dim, 2*N_uni)

def T(M, dim, N_uni):
    return np.trace(M) * np.eye(dim**(2*N_uni)) / (dim**(2*N_uni))

def Q_prime(M, dim, N_uni):
    MM = default2manysys(M, dim, 2*N_uni)
    pm = partial_red(MM, dim, 2*N_uni, [2*N_uni-1])-partial_red(MM, dim, 2*N_uni, [2*N_uni-1, 2*N_uni-2])
    for i in range(N_uni-1):
        if i != N_uni-2:
            pm += partial_red(MM, dim, 2*N_uni, list(range(2*N_uni-3-2*i, 2*N_uni))) \
            - partial_red(MM, dim, 2*N_uni, list(range(2*N_uni-4-2*i, 2*N_uni)))
        else:
            pm += partial_red(MM, dim, 2*N_uni, list(range(2*N_uni-3-2*i, 2*N_uni)))
    return manysys2default(pm, dim, 2*N_uni)

In [ ]:
def Omega_prime_proj(M, dim, N_uni):
    global Omega_global
    Omega_prime = normalize(Omega_global-Q_prime(Omega_global,dim,N_uni))
    return HS_ip(Omega_prime, M)*Omega_prime

def gen_B(dim, N_uni):
    global Omega_global
    H1 = np.random.uniform(-1.0, 1.0, (dim**(2*N_uni), dim**(2*N_uni)))
    H1 += H1.conj().T
    H2 = np.random.uniform(-1.0, 1.0, (dim**(2*N_uni), dim**(2*N_uni)))
    H2 += H2.conj().T
    H_1pr = H1 - Q_comb(H1,dim,N_uni)
    H_2pr = H2 - Q_comb(H2,dim,N_uni)
    a = HS_ip(Omega_global, H_1pr)
    b = np.trace(H_1pr)
    c = HS_ip(Omega_global, H_2pr)
    d = np.trace(H_2pr)
    fid = 0.45 #fidelity
    h1 = (fid*d-(dim**N_uni)*c)/(a*d-b*c)
    h2 = (-fid*b+(dim**N_uni)*a)/(a*d-b*c)
    B_pre = h1*H_1pr + h2*H_2pr
    return Q_prime(B_pre,dim,N_uni)+Omega_prime_proj(B_pre,dim,N_uni)

dim = 3
N_uni = 3

B_global = gen_B(dim, N_uni)

In [ ]:
print("trB:理論:{}, 実測:{}, B・Ω:理論:1, 実測:{}, Q_comb成分のノルム:理論:0, 実測:{}".format(dim**N_uni,np.trace(B_global),HS_ip(B_global,Omega_global), norm(Q_comb(B_global,dim,N_uni))))# dim**4, 1, 0

In [ ]:
def B_prime_proj(M, dim, N_uni):
    global B_global
    B_prime = normalize(B_global)
    return HS_ip(B_prime, M)*B_prime

def Q_(M, dim, N_uni):
    return Q_prime(M,dim,N_uni)+Omega_prime_proj(M,dim,N_uni)-B_prime_proj(M,dim,N_uni)

In [ ]:
def check333(dim, N_uni, rho_return=False, eps=(1e-6), imaj_ignr=1e-10, rtol=1e-10, pgtol=1e-20, m_=10, factr=1e2, comple=False, maxiter=10000):
    global t_global
    start = time.time()
    t_global = start
    if not comple:
        def Q(M):
            return Q_(M, dim, N_uni)
    if comple:
        def Q(M):
            return M - Q_(M, dim, N_uni)
    def f_mat(H):
        return norm(Q(H@H))**2 / (norm(H)**4)
    def G_mat(H):
        QHH = Q(H@H)
        n_Q = norm(QHH)
        n_H = norm(H)
        return (2/(n_H**4))*(H@QHH+QHH@H)-(4*n_Q**2/(n_H**6))*H

    def vec2mat(vec):
        return np.triu(vec.reshape((dim**(2*N_uni), dim**(2*N_uni)))) + \
            np.triu(vec.reshape((dim**(2*N_uni), dim**(2*N_uni)))).T + \
                1.0j*np.tril(vec.reshape((dim**(2*N_uni), dim**(2*N_uni)))) \
                -1.0j*np.tril(vec.reshape((dim**(2*N_uni), dim**(2*N_uni)))).T \
                -np.diag(vec.reshape((dim**(2*N_uni), dim**(2*N_uni))).diagonal())
    def f_vec(vec):
        H = vec2mat(vec)
        return f_mat(H).real
    def g_vec(vec):
        H = vec2mat(vec)
        G = G_mat(H)
        return (2 * (np.triu(G).real + np.tril(G).imag)- np.diag(G.diagonal().real)).reshape(dim**(4*N_uni))


    vec0 = np.eye(dim**(2*N_uni)).reshape(dim**(4*N_uni)) / (dim**N_uni)
    vec0 /= np.sqrt((vec0**2).sum())

    def lower_eig11(M, dim, N_uni, eps):
        assert M.shape == (dim**(2*N_uni), dim**(2*N_uni)), "invalid dimension"
        M1 = normalize(M)
        n = -int(np.ceil(np.log2(eps)))
        A = (np.eye(dim**(2*N_uni)) - M1)
        for i in range(n):
            AA = A@(A.conj().T)
            A = normalize(AA)
        return A
    def print_val(vec_now):
        global m_global
        global t_global
        H = vec2mat(vec_now)
        HH = H@H
        nor = np.sqrt(np.trace(HH))
        rho = HH/nor
        Qrho = Q(rho)
        Prho = rho-Qrho
        psipsi = lower_eig11(Qrho, dim, N_uni, eps=eps)
        innr = HS_ip(psipsi, Qrho)
        if (innr.real >= 0) and (abs(innr.imag) <= imaj_ignr):
            print("Q(rho) seems to be positive already")
            raise StopIteration
        psi_comp = lower_eig11(Prho, dim, N_uni, eps=eps)
        innr_comp = HS_ip(psi_comp, Prho)
        if (innr_comp.real >= 0) and (abs(innr_comp.imag) <= imaj_ignr):
            print("P(rho) seems to be positive already")
            raise StopIteration
        v = f_mat(H).real
        if v <= rtol / (dim**(N_uni)):
            print("v*(dim**(N_uni)):{:.4e}, smaller than {:.2e}".format(v*(dim**(N_uni)), rtol))
            raise StopIteration
        tnow = time.time()
        print("m:{},val:{:.4e},norm:{:.2f},eig_m:{:.2e},t_abs:{:.0f},t_dif:{:.0f} ".format(m_global,v,nor,innr.real,tnow-start,tnow-t_global))
        t_global = tnow
        m_global += 1
        return True
    vec_min, v, dic = optimize.fmin_l_bfgs_b(f_vec, vec0, fprime=g_vec,callback=print_val,m=m_,pgtol=pgtol,factr=factr, maxiter=maxiter)
    m = dic["nit"]

    if v > rtol / (dim**(N_uni)):
        print("rho exists(F(rho)*(dim**(N_uni)):{:.4e}), m:{}".format(v*(dim**(N_uni)),m))
    if v <= rtol / (dim**(N_uni)):
        print("rho does not exist(F(rho)*(dim**(N_uni)):{:.4e}), m:{}".format(v*(dim**(N_uni)),m))
    H = vec2mat(vec_min)
    rho = H@H/np.trace(H@H)
    if rho_return:
        return rho
    if not rho_return:
        val = np.linalg.eigvals(Q(rho))
        min_val = np.min(val)
        if v > rtol / (dim**(N_uni)):
            return (m, True, min_val)
        if v <= rtol / (dim**(N_uni)):
            return (m, False, min_val)

In [ ]:
m_global = 0 #カウンタの変数("m_global"という名前じゃないといけない)
t_global = 0

rho = check333(dim, N_uni, rho_return=True, m_=10, factr=1e-2, comple=True)

In [ ]:
is_pos_def(rho)